In [1]:
!pip install langchain langchain_community langchain_google_genai "google-generativeai>=0.7.2"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.7 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',verbose=True)

In [5]:
import google.generativeai as genai


def ac_turn_on():
  """Turning on the AC"""
  print("AC Turned On!")

def increase_temp():
    """Increasing AC temperature"""
    print("Temperature Increased!")

def decrease_temp():
  """Decreasing AC temperature"""
  print("Temperature Decreased!")

def ac_turn_off():
  """Turning off the AC"""
  print("AC Turned Off!")

ac_controls = [ac_turn_on, increase_temp, decrease_temp, ac_turn_off]
instruction = "Your are a helpful assistant who controls the turning on, turning off, increasing and decreasing the temperature of an AC. You will not do anything else."

model = genai.GenerativeModel(
    "models/gemini-1.5-pro", tools=ac_controls, system_instruction=instruction
)

chat = model.start_chat()


In [6]:
response = chat.send_message("Turn on the AC")
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "ac_turn_on",
                  "args": {}
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP"
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 138,
        "candidates_token_count": 5,
        "total_token_count": 143
      }
    }),
)

In [7]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

In [8]:
available_fns = ["ac_turn_on", "increase_temp", "decrease_temp", "ac_turn_off"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("Turn on the AC!", tool_config=tool_config)
print(response.parts[0])

function_call {
  name: "ac_turn_on"
  args {
  }
}



In [9]:
auto_chat = model.start_chat(enable_automatic_function_calling=True)
auto_chat.send_message("Turn on the AC and increase the temperature", tool_config=tool_config)

AC Turned On!
Temperature Increased!


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "OK. I've turned on the AC and increased the temperature. Anything else?\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.03560457958115472
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 196,
        "candidates_token_count": 18,
        "total_token_count": 214
      }
    }),
)

In [10]:
from langchain.tools import Tool

# Example tool for addition
def add(a, b):
    return a + b

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

def subtract(a: int, b: int) -> int:
    """Subtract b from a.

    Args:
        a: first int
        b: second int
    """
    return a - b

def divide(a: int, b: int) -> int:
    """Divide a by b.

    Args:
        a: first int
        b: second int
    """
    return a / b

add_tool = Tool(
    name="AdditionTool",
    func=lambda x: f"The result is: {add(*map(int, x.split()))}",
    description="Adds two numbers. Input should be two integers separated by a space."
)

multiply_tool = Tool(
    name="MulitplicationTool",
    func=lambda x: f"The result is: {multiply(*map(int, x.split()))}",
    description="Mulitplies two numbers. Input should be two integers separated by a space."
)

subtract_tool = Tool(
    name="SubtractionTool",
    func=lambda x: f"The result is: {subtract(*map(int, x.split()))}",
    description="Subtracts smaller number from the bigger number. Input should be two integers separated by a space."
)

divide_tool = Tool(
    name="DivisionTool",
    func=lambda x: f"The result is: {divide(*map(int, x.split()))}",
    description="Divides two numbers. Division should not be possible if second input is 0. Input should be two integers separated by a space."
)

In [11]:
from langchain.agents import initialize_agent, AgentType

# List of tools
tools = [add_tool, multiply_tool, subtract_tool, divide_tool]

# Initialize the agent
agent = initialize_agent(
    tools=tools,                        # Provide the tools
    llm=llm,                            # LLM for fallback
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent type
    verbose=True                        # Enable debugging output
)

<ipython-input-11-8fbfc3fecaa4>:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [12]:
response1 = agent.run("What is 5 * 20?")  # Uses the tool
print(response1)

<ipython-input-12-c2eaaac76e86>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response1 = agent.run("What is 5 * 20?")  # Uses the tool




> Entering new AgentExecutor chain...
Question: What is 5 * 20?
Thought: I need to multiply 5 and 20.
Action: MulitplicationTool
Action Input: 5 20
Observation: The result is: 100
Thought:Question: What is 5 * 20?
Thought: I need to multiply 5 and 20.
Action: MulitplicationTool
Action Input: 5 20
Observation: The result is: 100
Thought:Question: What is 5 * 20?
Thought: I need to multiply 5 and 20.
Action: MulitplicationTool
Action Input: 5 20
Observation: The result is: 100
Thought:Question: What is 5 * 20?
Thought: I need to multiply 5 and 20.
Action: MulitplicationTool
Action Input: 5 20
Observation: The result is: 100
Thought:Question: What is 5 * 20?
Thought: I need to multiply 5 and 20.
Action: MulitplicationTool
Action Input: 5 20
Observation: The result is: 100
Thought:Question: What is 5 * 20?
Thought: I need to multiply 5 and 20.
Action: MulitplicationTool
Action Input: 5 20
Observation: The result is: 100
Thought:Question: What is 5 * 20?
Thought: I need to multiply 5 and 